# Another SSD model
more adapted for small objects

In [1]:
import sys
from pathlib import Path

root_path = Path('..').absolute().resolve()
if root_path.as_posix() not in sys.path:
    sys.path.append(root_path.as_posix())

In [2]:
import torch
from models.net import AnotherSSD300, BoxCoder300

In [3]:
model = AnotherSSD300(2)

In [4]:
model = model.cuda()

In [5]:
from torch.autograd import Variable

x = Variable(torch.rand(1, 3, 300, 300).cuda(), requires_grad=True)

In [6]:
feature_maps = model.extractor(x)

In [7]:
for fm in feature_maps:
    print(fm.shape)

torch.Size([1, 64, 75, 75])
torch.Size([1, 128, 37, 37])
torch.Size([1, 256, 18, 18])


In [8]:
for index in range(len(feature_maps)):
    loc_pred = model.loc_layers[index](feature_maps[index])
    loc_pred = loc_pred.permute(0,2,3,1).contiguous()
    loc_pred = loc_pred.view(loc_pred.size(0),-1,4)
    print("Output loc pred {} :".format(index), loc_pred.shape)

Output loc pred 0 : torch.Size([1, 5776, 4])
Output loc pred 1 : torch.Size([1, 1444, 4])
Output loc pred 2 : torch.Size([1, 324, 4])


In [9]:
fm_sizes = []
xs = feature_maps
for i, _x in enumerate(xs):
    loc_pred = model.loc_layers[i](_x)
    fm_sizes.append(loc_pred.shape)
fm_sizes

[torch.Size([1, 16, 38, 38]),
 torch.Size([1, 16, 19, 19]),
 torch.Size([1, 16, 9, 9])]

In [10]:
y = model(x)

In [11]:
y[0].shape

torch.Size([1, 7544, 4])

In [12]:
box_coder = BoxCoder300(feature_map_sizes=model.feature_map_sizes)

In [13]:
len(box_coder.default_boxes)

7544

In [14]:
from customized_torchcv.loss.ssd_loss import SSDLoss
from ignite._utils import to_variable

In [15]:
loss_fn = SSDLoss(2)

In [16]:
objs = torch.Tensor([[1, 1, 5, 5], [10, 10, 11, 11], [150, 150, 155, 155]])
labels = torch.LongTensor([1, 0, 2])

loc_targets, cls_targets = box_coder.encode(objs, labels)
loc_targets = loc_targets.unsqueeze(0).cuda()
cls_targets = cls_targets.unsqueeze(0).cuda()
y_true = loc_targets, cls_targets

In [17]:
cls_targets.shape, loc_targets.shape

(torch.Size([1, 7544]), torch.Size([1, 7544, 4]))

In [18]:
loss_fn(y, to_variable(y_true))

Variable containing:
 13.4976
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [82]:
feature_maps = [None] * 5

def get_feature_map(index):
    def hook(module, input, result):
        feature_maps[index] = result
    return hook

In [84]:
vgg_features = model.features
# indices = [5, 12, 22, 32, 42]
# for i in indices:
#     print(vgg_features[i])

In [37]:
handles = []
for i, index in enumerate(indices):
    handles.append(vgg_features[index].register_forward_hook(get_feature_map(i)))

In [38]:
from torch.autograd import Variable

x = Variable(torch.rand(2, 3, 128, 128), requires_grad=True)

In [42]:
print(feature_maps)
_ = vgg_features(x)

[None, None, None, None, None]


In [43]:
for fm in feature_maps:
    print(fm.shape)

torch.Size([2, 64, 128, 128])
torch.Size([2, 128, 64, 64])
torch.Size([2, 256, 32, 32])
torch.Size([2, 512, 16, 16])
torch.Size([2, 512, 8, 8])
